## Testing Submodular Function Maximization Algorithms

In [2]:
import torch
from src.constraint import Cardinality, PartitionMatroid
from src.dataset import MovieLensDataset
from src.selector import (
    FWSelector,
    GreedySelector,
    RandomSelector,
    SGASelector,
    SmoothedGreedySelector,
)
from src.utility import InfluenceUtility

torch.manual_seed(0)

In [3]:
N_INSTANCES = 1
N_MOVIES = 100
N_USERS = 500
RANDOM_STATE = 42
DATA_DIR = "./ml-100k-data"
DATA_ZIP_PATH = f"{DATA_DIR}/ml-100k.zip"
EXTRACTED_PATH = f"{DATA_DIR}/ml-100k"

In [4]:
dataset = MovieLensDataset(
    DATA_DIR,
    DATA_ZIP_PATH,
    n_instances=N_INSTANCES,
    n_movies=N_MOVIES,
    n_users=N_USERS,
    random_state=RANDOM_STATE,
)
print(dataset.theta.sum())

Dataset already exists.
Num Movies: 1682, Num Users: 943, Num Ratings:100000


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 50.65it/s]

tensor(227.)


In [5]:
dataset[0][0].shape

torch.Size([100, 500, 43])

## Under the Cardinality Constraint

### Greedy, SGA, FW, and SmoothedGreedy perform similarly and are better than Random

In [6]:
def eval_all(theta: torch.Tensor, utility, constraint):
    torch.manual_seed(RANDOM_STATE)
    random = RandomSelector(utility, constraint)
    S = random(theta)
    print(f"    Random: {torch.mean(utility(S)).item():.3f}")

    greedy = GreedySelector(utility, constraint)
    S = greedy(theta)
    print(f"    Greedy: {torch.mean(utility(S)).item():.3f}")

    sga = SGASelector(utility, constraint, max_epochs=20)
    S = sga(theta)
    print(f"       SGA: {torch.mean(utility(S)).item():.3f}")

    fw = FWSelector(utility, constraint, delta=0.05)
    S = fw(theta)
    print(f"        FW: {torch.mean(utility(S)).item():.3f}")

    smooth_greedy = SmoothedGreedySelector(utility, constraint, epsilon=0.1)
    smooth_greedy.eval()
    S = smooth_greedy(theta)
    print(f"   SmoothG: {torch.mean(utility(S)).item():.3f}")

In [7]:
theta = dataset.theta

In [8]:
for k in [5, 10, 20, 40]:
    print(f"----- k = {k} -----")
    utility = InfluenceUtility()
    constraint = Cardinality(max_cardinality=k)
    eval_all(theta, utility, constraint)

----- k = 5 -----
    Random: 32.694
    Greedy: 74.960
       SGA: 74.960
        FW: 74.960
   SmoothG: 74.960
----- k = 10 -----
    Random: 39.549
    Greedy: 105.532
       SGA: 105.532
        FW: 105.532
   SmoothG: 105.256
----- k = 20 -----
    Random: 64.790
    Greedy: 134.378
       SGA: 134.378
        FW: 134.306
   SmoothG: 134.357
----- k = 40 -----
    Random: 103.151
    Greedy: 156.005
       SGA: 156.005
        FW: 155.914
   SmoothG: 155.369


### Compare with brute-force (optimal solution) on a small MovieLens Dataset

In [9]:
import itertools


def brute_force(utility, all_candidates):
    n_items = utility.get_n_items()
    combinations = []
    opt = torch.tensor(-torch.inf)
    batch_size = 10000

    def _batch_eval(opt):
        S_all = torch.zeros((batch_size, n_items))
        _combinations = torch.tensor(combinations)
        S_all.scatter_(1, _combinations, 1.0)
        utility_all = utility(S_all)
        argmax = torch.argmax(utility_all)
        S_opt = torch.clone(S_all[argmax])
        opt = torch.maximum(opt, utility_all[argmax])
        return opt, S_opt

    for indices in all_candidates:
        combinations.append(indices)
        if len(combinations) == batch_size:
            opt, _ = _batch_eval(opt)
            combinations = []
    if combinations:
        opt, _ = _batch_eval(opt)
    return opt

In [10]:
dataset_small = MovieLensDataset(
    DATA_DIR,
    DATA_ZIP_PATH,
    n_instances=1,
    n_movies=30,
    n_users=N_USERS,
    random_state=RANDOM_STATE,
)
theta_small = dataset_small[0][1].view(1, 30, N_USERS)

Dataset already exists.
Num Movies: 1682, Num Users: 943, Num Ratings:100000


100%|██████████| 1/1 [00:00<00:00, 81.00it/s]


In [11]:
from math import exp

for k in [1, 2, 3, 4, 5]:
    print(f"----- k = {k} -----")
    utility = InfluenceUtility()
    constraint = Cardinality(max_cardinality=k)
    utility.set_params(theta_small)
    all_candidates = itertools.combinations(range(utility.get_n_items()), r=k)
    optimal = brute_force(utility, all_candidates)
    print(f"   Optimal: {optimal:.3f}")
    print(f"(1-1/e)Opt: {(1 - 1.0/exp(1))*optimal:.3f}")
    eval_all(theta_small, utility, constraint)

----- k = 1 -----
   Optimal: 27.580
(1-1/e)Opt: 17.434
    Random: 27.580
    Greedy: 27.580
       SGA: 27.580
        FW: 27.580
   SmoothG: 27.580
----- k = 2 -----
   Optimal: 39.097
(1-1/e)Opt: 24.714
    Random: 27.774
    Greedy: 39.097
       SGA: 39.097
        FW: 39.097
   SmoothG: 39.097
----- k = 3 -----
   Optimal: 47.441
(1-1/e)Opt: 29.989
    Random: 28.192
    Greedy: 47.441
       SGA: 47.441
        FW: 47.441
   SmoothG: 47.441
----- k = 4 -----
   Optimal: 55.012
(1-1/e)Opt: 34.774
    Random: 31.301
    Greedy: 55.012
       SGA: 55.012
        FW: 55.012
   SmoothG: 55.012
----- k = 5 -----
   Optimal: 61.082
(1-1/e)Opt: 38.611
    Random: 39.947
    Greedy: 61.082
       SGA: 61.082
        FW: 61.082
   SmoothG: 61.082


## Under the Partition Matroid Constraint

### Greedy, SGA, FW, and SmoothedGreedy perform similarly and are better than Random

In [12]:
theta = dataset[0][1].view(1, N_MOVIES, N_USERS)
n_partition = 5
partition = [
    [j + i * (N_MOVIES // n_partition) for j in range(N_MOVIES // n_partition)]
    for i in range(n_partition)
]

for k in [1, 2, 3, 4]:
    max_cardinalities = [k] * n_partition
    print(f"----- k = {k} -----")
    utility = InfluenceUtility()
    constraint = PartitionMatroid(
        max_cardinalities=max_cardinalities, partition=partition
    )
    eval_all(theta, utility, constraint)

----- k = 1 -----
    Random: 31.222
    Greedy: 66.597
       SGA: 66.597
        FW: 66.597
   SmoothG: 66.597
----- k = 2 -----
    Random: 38.648
    Greedy: 96.470
       SGA: 96.470
        FW: 96.470
   SmoothG: 96.470
----- k = 3 -----
    Random: 50.912
    Greedy: 116.726
       SGA: 116.971
        FW: 116.726
   SmoothG: 116.726
----- k = 4 -----
    Random: 55.958
    Greedy: 131.233
       SGA: 131.233
        FW: 131.233
   SmoothG: 131.233


In [13]:
theta = torch.rand(1, N_MOVIES, N_USERS) * 0.2
mask = torch.rand(1, N_MOVIES, N_USERS) > 0.95
theta *= mask
n_partition = 5
partition = [
    [j + i * (N_MOVIES // n_partition) for j in range(N_MOVIES // n_partition)]
    for i in range(n_partition)
]

for k in [1, 2, 3, 4]:
    max_cardinalities = [k] * n_partition
    print(f"----- k = {k} -----")
    utility = InfluenceUtility()
    constraint = PartitionMatroid(
        max_cardinalities=max_cardinalities, partition=partition
    )
    eval_all(theta, utility, constraint)

----- k = 1 -----
    Random: 10.458
    Greedy: 17.507
       SGA: 17.500
        FW: 17.500
   SmoothG: 17.448
----- k = 2 -----
    Random: 21.251
    Greedy: 33.152
       SGA: 33.152
        FW: 33.152
   SmoothG: 32.736
----- k = 3 -----
    Random: 32.583
    Greedy: 47.557
       SGA: 47.557
        FW: 47.557
   SmoothG: 47.351
----- k = 4 -----
    Random: 43.988
    Greedy: 60.933
       SGA: 60.916
        FW: 60.863
   SmoothG: 60.578


### Compare with brute-force (optimal solution) on a small MovieLens Dataset

In [14]:
from functools import reduce


def all_partition_matroid_candidates(max_cardinalities, partition):
    iterators = []
    for k, p in zip(max_cardinalities, partition):
        iterators.append(itertools.combinations(p, r=k))
    for indices in itertools.product(*iterators):
        yield reduce(lambda x, y: x + y, indices)

In [15]:
dataset_small = MovieLensDataset(
    DATA_DIR,
    DATA_ZIP_PATH,
    n_instances=1,
    n_movies=35,
    n_users=N_USERS,
    random_state=RANDOM_STATE,
)
theta_small = dataset_small[0][1].view(1, 35, N_USERS)

Dataset already exists.
Num Movies: 1682, Num Users: 943, Num Ratings:100000


100%|██████████| 1/1 [00:00<00:00, 79.93it/s]


In [18]:
n_partition = 7
partition = [
    [
        j + i * (theta_small.shape[1] // n_partition)
        for j in range(theta_small.shape[1] // n_partition)
    ]
    for i in range(n_partition)
]
for k in [1, 2, 3]:
    print(f"----- k = {k} -----")
    utility = InfluenceUtility()
    max_cardinalities = [k] * n_partition
    constraint = PartitionMatroid(
        max_cardinalities=max_cardinalities, partition=partition
    )
    utility.set_params(theta_small)
    all_candidates = all_partition_matroid_candidates(max_cardinalities, partition)
    optimal = brute_force(utility, all_candidates)
    print(f"   Optimal: {optimal:.3f}")
    print(f"(1-1/e)Opt: {(1 - 1.0/exp(1))*optimal:.3f}")
    eval_all(theta_small, utility, constraint)

----- k = 1 -----
   Optimal: 65.048
(1-1/e)Opt: 41.118
    Random: 38.391
    Greedy: 65.048
       SGA: 65.048
        FW: 65.048
   SmoothG: 65.048
----- k = 2 -----
   Optimal: 80.324
(1-1/e)Opt: 50.775
    Random: 60.154
    Greedy: 80.324
       SGA: 80.324
        FW: 80.324
   SmoothG: 80.135
----- k = 3 -----
   Optimal: 88.026
(1-1/e)Opt: 55.643
    Random: 75.356
    Greedy: 88.026
       SGA: 88.026
        FW: 88.026
   SmoothG: 87.910
